In [9]:
import os
import string
import glob
import shutil
import re, random


import h5py

import matplotlib.pyplot as plt
import time, pickle, pandas

import numpy as np

import keras
from keras.optimizers import rmsprop
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential, load_model
from keras.layers import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.layers import BatchNormalization
from keras.callbacks import TensorBoard, ModelCheckpoint
from keras import backend
from keras import optimizers
from keras import backend as K

import tensorflow as tf

import utils
reload(utils)

%matplotlib inline

In [49]:
utils.take_sample('../data/tu-berlin',
            '../data/tu-berlin/sample50', 50)

In [2]:
batch_size = 10

data_root = '../data/tu-berlin/sample50/'
train_data_dir = data_root + 'train'
validation_data_dir = data_root + 'test'



# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.25,
        zoom_range=0.25,
        rotation_range=35,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
#         target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
#         target_size=(img_width, img_height),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode='grayscale')

batch, y = train_generator.next()
img_height = batch.shape[1]
img_width = batch.shape[2]
channels = batch.shape[3]
num_classes = len(y[0])
print img_height, img_width, channels, num_classes

Found 2650 images belonging to 50 classes.
Found 1350 images belonging to 50 classes.
256 256 1 50


In [3]:
model_name = 'sketchnet2'
%mkdir -p ../logs/{model_name}
%mkdir -p ../models/{model_name}

log_cb =\
    TensorBoard(log_dir='../logs/'+model_name+'/', 
                histogram_freq=0, 
                write_graph=False, write_images=False)
best_model_cb =\
    ModelCheckpoint('../models/'+model_name+'/best.h5', 
                    monitor='val_acc', verbose=0, 
                    save_best_only=True, 
                    mode='auto', period=1)
latest_model_cb =\
    ModelCheckpoint('../models/'+model_name+'/latest.h5', 
                    monitor='val_acc', verbose=0, 
                    period=1)

In [4]:
def build_sketchnet1():
    model = Sequential()
    model.add(Convolution2D(64, (3, 3), 
                            padding='same',
                            activation='relu',
                            input_shape=(img_width, img_height, channels)))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(128, (3, 3), 
                            padding='same', 
                            activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.125))

    model.add(Convolution2D(256, (3, 3), 
                            padding='same', 
                            activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(3, activation='softmax'))

    # initialize optimizer
    opt = rmsprop(lr=0.0001)

#     model.compile(loss='categorical_crossentropy',
#                   optimizer=opt,
#                   metrics=['accuracy'])
    
    return model, opt

model, opt = build_sketchnet1()
model.load_weights('../models/sketchnet1_best.h5')

In [5]:
model.layers.pop()
model.layers[-1].outbound_nodes = []
model.outputs = [model.layers[-1].output]
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 64)      640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 128)     73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 64, 64, 128)       0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 64, 64, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 256)       295168    
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 32, 32, 256)       0         
__________

In [6]:
total_epochs=0

In [7]:
nb_epoch = 1
K.set_value(opt.lr, 0.0001)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
#         nb_val_samples = nb_validation_samples,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb, 
                   best_model_cb,
                   latest_model_cb]
)
total_epochs += nb_epoch

Epoch 1/1
265/265 [==============================] - 95s - loss: 3.9070 - acc: 0.0321 - val_loss: 3.8329 - val_acc: 0.0556


In [ ]:
nb_epoch = 25
K.set_value(opt.lr, 0.001)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
#         nb_val_samples = nb_validation_samples,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb, 
                   best_model_cb,
                   latest_model_cb]
)
total_epochs += nb_epoch

Epoch 2/26
265/265 [==============================] - 111s - loss: 3.5669 - acc: 0.0947 - val_loss: 2.9160 - val_acc: 0.2622
Epoch 3/26
265/265 [==============================] - 110s - loss: 3.0264 - acc: 0.1985 - val_loss: 2.5593 - val_acc: 0.3178
Epoch 4/26
265/265 [==============================] - 110s - loss: 2.8600 - acc: 0.2392 - val_loss: 2.4103 - val_acc: 0.3667
Epoch 5/26
265/265 [==============================] - 110s - loss: 2.7268 - acc: 0.2774 - val_loss: 2.3099 - val_acc: 0.3993
Epoch 6/26
264/265 [============================>.] - ETA: 0s - loss: 2.5769 - acc: 0.3042

In [11]:
metrics = model.evaluate_generator(train_generator, steps=265)
print 'val', model.metrics_names, metrics
metrics = model.evaluate_generator(validation_generator, steps=135)
print 'val', model.metrics_names, metrics

 val ['loss', 'acc'] [1.6908005754902677, 0.55773586434013434]
val ['loss', 'acc'] [2.2257320075123399, 0.50444445599008492]


In [ ]:
nb_epoch = 25
K.set_value(opt.lr, 0.0001)

hist = model.fit_generator(
        train_generator,
        steps_per_epoch = 265,
        epochs = nb_epoch + total_epochs,
        validation_data = validation_generator,
        validation_steps = 135,
#         nb_val_samples = nb_validation_samples,
        verbose = 1,
        initial_epoch = total_epochs,
        callbacks=[log_cb, 
                   best_model_cb,
                   latest_model_cb]
)
total_epochs += nb_epoch

Epoch 27/51
265/265 [==============================] - 110s - loss: 1.9351 - acc: 0.4777 - val_loss: 1.9187 - val_acc: 0.5430
Epoch 28/51
265/265 [==============================] - 100s - loss: 1.8514 - acc: 0.5000 - val_loss: 1.9673 - val_acc: 0.5385
Epoch 29/51
265/265 [==============================] - 100s - loss: 1.8923 - acc: 0.4992 - val_loss: 1.9312 - val_acc: 0.5363
Epoch 30/51
265/265 [==============================] - 110s - loss: 1.8759 - acc: 0.5106 - val_loss: 1.8682 - val_acc: 0.5541
Epoch 31/51
265/265 [==============================] - 100s - loss: 1.8553 - acc: 0.4985 - val_loss: 1.9738 - val_acc: 0.5415
Epoch 32/51
265/265 [==============================] - 100s - loss: 1.8406 - acc: 0.5053 - val_loss: 1.8971 - val_acc: 0.5378
Epoch 36/51
265/265 [==============================] - 101s - loss: 1.8398 - acc: 0.5321 - val_loss: 1.8486 - val_acc: 0.5585
Epoch 40/51
265/265 [==============================] - 100s - loss: 1.8274 - acc: 0.5204 - val_loss: 1.8390 - val_acc:

In [ ]:
metrics = model.evaluate_generator(train_generator, steps=265)
print 'train', model.metrics_names, metrics
metrics = model.evaluate_generator(validation_generator, steps=135)
print 'val', model.metrics_names, metrics